In [ ]:
# test the use of runlength encoding for rainfields rainfall data 
# a field is identified using variable, stn_id, and valid_time
# each stn_id has only one projection 
# TO DO change this when working with forecasts where we get many forecasts for a valid time 

In [ ]:
from pymongo import MongoClient
from netCDF4 import Dataset 
import datetime
import numpy as np 
import bson
from bson.binary import Binary
import base64
import run_len

In [ ]:
# open the nc file and read in the stn_id and valid time
rf3_name = "/home/awseed/data/RF3/2/20190322/2_20190322_235500.prcp-c5.nc"
#rf3_name="/home/awseed/data/RF3/prcp-crate/2021/20210129/310_20210129_003000.prcp-crate.nc" 
rf3 = Dataset(rf3_name)
stn_id = int(rf3.getncattr("station_id"))
time_stamp = int(rf3['valid_time'][:])
valid_time = datetime.datetime.utcfromtimestamp(time_stamp) 

if "start_time" in rf3.variables.keys():
    time_stamp = int(rf3['start_time'][:])
    start_time = datetime.datetime.utcfromtimestamp(time_stamp) 
    rain_variable = "precipitation"
else:
    start_time = 0 
    rain_variable = "rain_rate"


In [ ]:
# open the database and collections
client = MongoClient()
db = client["test_2"]
db.drop_collection("projection")
db.drop_collection("valid_time")
db.drop_collection("data")

proj_col = db["projection"] 
vt_col   = db["valid_time"]
rr_col   = db["data"]

In [ ]:
# check if we have a copy of this field in the database 
count_fields = vt_col.count_documents(filter={"stn_id": stn_id, "valid_time":valid_time}) 

In [ ]:
if count_fields == 0:

    # read the valid_time attrubutes from the nc file and insert into database 
    valid_time_attr = rf3.variables['valid_time'].__dict__ 
    valid_time_attr['stn_id'] = stn_id 
    valid_time_attr['variable'] = rain_variable
    valid_time_attr['valid_time'] = valid_time
    valid_time_attr['start_time'] = start_time
    vt_id = vt_col.insert_one(valid_time_attr).inserted_id

    # get the projection details
    proj = {}
    count_docs = proj_col.count_documents(filter={"stn_id": stn_id})
    if count_docs > 0:
        # get the projection from the database 
        proj = proj_col.find_one({"stn_id": stn_id}) 
    else:
        
        # get the projection from the ncfile and write it to the database 
        proj_attr = {}
        proj_attr['grid_mapping_name'] = rf3.variables['proj'].getncattr("grid_mapping_name") 
        standard_parallel = rf3.variables['proj'].getncattr("standard_parallel")
        proj_attr['standard_parallel'] =  standard_parallel.tolist()
        proj_attr['longitude_of_central_meridian'] = rf3.variables['proj'].getncattr("longitude_of_central_meridian")
        proj_attr['latitude_of_projection_origin'] = rf3.variables['proj'].getncattr("latitude_of_projection_origin")
        proj_attr['false_easting'] = rf3.variables['proj'].getncattr("false_easting") 
        proj_attr['false_northing'] = rf3.variables['proj'].getncattr("false_northing") 
        
        if "towgs84" in rf3.variables['proj'].__dict__ :
            towgs84 = rf3.variables['proj'].getncattr("towgs84")
            proj_attr['towgs84'] = towgs84.tolist()
        
        x_attr = rf3.variables['x'].__dict__ 
        x = rf3['x'][:]
        x_attr['x'] = Binary(x) 
        
        y_attr = rf3.variables['y'].__dict__ 
        y = rf3['y'][:]
        y_attr['y'] = Binary(y) 
        
        proj['stn_id'] = stn_id
        proj['proj'] = proj_attr 
        proj['y'] = y_attr
        proj['x'] = x_attr

        proj_id = proj_col.insert_one(proj).inserted_id

In [ ]:
    # read in metadata for rainfall and write it out 
    rain_attr = rf3.variables[rain_variable].__dict__ 
    rain_attr['stn_id'] = int(stn_id) 
    rain_attr['variable'] = rain_variable
    rain_attr['valid_time'] = valid_time
    rain_attr['_FillValue'] = int(rain_attr['_FillValue'])

    rain = rf3[rain_variable][:] / 0.05 

    rle_rain = run_len.encode(rain) 
    rain_attr[rain_variable] = rle_rain
    rr_id = rr_col.insert_one(rain_attr).inserted_id

In [ ]:
# test reading the rainfall record 
test_rain = rr_col.find_one({"valid_time":valid_time,"stn_id": stn_id}) 

In [ ]:
precip = test_rain['precipitation'] 
print(precip)